# Heat Case Study - Energy System Model

In [ ]:
import os

import pandas as pd

import oemof.tabular.facades as fc
from oemof.solph import EnergySystem, Model, Bus
from oemof.tabular.tools import postprocessing as pp

results_path = "results"
if not os.path.exists(results_path): 
    os.mkdir(results_path)

In [ ]:
timeseries = pd.read_csv("data/timeseries.csv", parse_dates=True, index_col=0)

heat_load_max = timeseries["heat_load"].max() 

timeseries.index.freq = "H"

In [ ]:
es = EnergySystem(timeindex=timeseries.index)

## Add Components to energy system

### Bus Constraint

With the set of all Buses $B$ all inputs $x^{flow}_{i(b),b}$ to a bus $b$ must equal all its outputs $x^{flow}_{b,o(b)}$

$$\sum_i x^{flow}_{i(b), b}(t) - \sum_o x^{flow}_{b, o(b)}(t) = 0 \qquad \forall t \in T, \forall b \in B$$

This equation will be build once the complete energy system is setup with its component. Every time a `Component` is created, the connected bus inputs/outputs will be updated. By this update every bus has all required information of its inputs and outputs available to construct the constraints. 


In [ ]:
elec_bus = Bus(label="elec_bus")
heat_bus = Bus(label="heat_bus")
gas_bus = Bus(label="gas_bus")

es.add(elec_bus, heat_bus, gas_bus)



### Commodity

The commodity object will bound the flow from a source to a bus by a certain `amount` by the following equation:

$$ \sum_{t} x^{flow}(t) \leq c^{amount} \qquad for t \in T$$



In [ ]:
es.add(
    fc.Commodity(
        label="gas-commdity",
        bus=gas_bus,
        amount=10e10,
        carrier="gas",
        tech="commodity",
        marginal_cost=25,
    )
)

### Extraction Turbine Unit

The mathematical description is derived from the oemof base class
[ExtractionTurbineCHP](https://oemof.readthedocs.io/en/
stable/oemof_solph.html#extractionturbinechp-component>):

$$x^{flow, carrier}(t) =
    \frac{x^{flow, electricity}(t) + x^{flow, heat}(t) \
    \cdot c^{beta}(t)}{c^{condensing\_efficiency}(t)}
    \qquad \forall t \in T$$

$$
    x^{flow, electricity}(t)  \geq  x^{flow, thermal}(t) \cdot
    \frac{c^{electrical\_efficiency}(t)}{c^{thermal\_efficiency}(t)}
    \qquad \forall t \in T$$

where $c^{beta}$ is defined as:

$$
    c^{beta}(t) = \frac{c^{condensing\_efficiency}(t) -
    c^{electrical\_efficiency(t)}}{c^{thermal\_efficiency}(t)}
    \qquad \forall t \in T$$

**Ojective expression** for operation includes marginal cost and/or
carrier costs:

$$
        x^{opex} = \sum_t (x^{flow, out}(t) \cdot c^{marginal\_cost}(t)
        + x^{flow, carrier}(t) \cdot c^{carrier\_cost}(t))
$$


In [ ]:


es.add(
    fc.ExtractionTurbine(
        label="extraction-turbine",
        electricity_bus=elec_bus,
        heat_bus=heat_bus,
        fuel_bus=gas_bus,
        carrier="gas",
        tech="ext",
        capacity=heat_load_max * 0.2,
        condensing_efficiency=0.55,
        electric_efficiency=0.4,
        thermal_efficiency=0.3
    )
)





### Backpressur Units / Motoric CHP 

Backpressure turbine power plants are modelled with a constant relation
between heat and electrical output (power to heat coefficient).

$$ x^{flow, carrier}(t) =
    \frac{x^{flow, electricity}(t) + x^{flow, heat}(t)}\
    {c^{thermal\:efficiency}(t) + c^{electrical\:efficiency}(t)}
    \qquad \forall t \in T $$


$$    \frac{x^{flow, electricity}(t)}{x_{flow, thermal}(t)} =
    \frac{c^{electrical\:efficiency}(t)}{c^{thermal\:efficiency}(t)}
    \qquad \forall t \in T $$

**Ojective expression** for operation includes marginal cost and/or
carrier costs:

$$        x^{opex} = \sum_t (x^{flow, out}(t) \cdot c^{marginal\_cost}(t)
        + x^{flow, carrier}(t) \cdot c^{carrier\_cost}(t))$$

In [ ]:
es.add(
    fc.BackpressureTurbine(
        label="motoric-chp",
        electricity_bus=elec_bus,
        heat_bus=heat_bus,
        fuel_bus=gas_bus,
        carrier="gas",
        tech="bp",
        capacity=heat_load_max * 0.2,
        electric_efficiency=0.4,
        thermal_efficiency=0.4
    )
)


### Heat Load

For the set of all Load denoted with $l \in L$ the load $x_l$ at timestep t equals the exogenously defined  profile value $c^{profile}_l$ multiplied by the amount of this load $c^{amount}_l$

$$ x^{flow}_{l}(t) = c^{profile}_{l}(t) \cdot c^{amount}_{l} \qquad \forall t \in T, \forall l \in L$$

In [ ]:

es.add(
    fc.Load(
        label="heat-load",
        bus=heat_bus,
        amount=1145.1273e3,
        profile=timeseries["heat_load_profile"]))

es.add(
    fc.Excess(
        label="grid",
        bus=elec_bus,
        marginal_cost=-1 * timeseries["electricity_price"].values))

### Storage

The mathematical representation of the storage for all storages $s \in S$ will include the flow into the storage, out of the storage and a storage level. The defaul efficiency for input/output is 1. Note that this is is included during charge and discharge. If you want to set the round trip efficiency you need to do for example: $\eta = \sqrt{\eta^{roundtrip}}$

Intertemporal energy balance of the storage:

$$ x^{level}_{s}(t) = \eta^{loss} x^{level}_{s}(t) + \eta x^{flow}_{s, in} - \eta x^{flow}_{s, out}(t) \qquad \forall t \in T,  \forall s \in S$$ 

Bounds of the storage level variable $x^{level}_s(t)$:

$$ x^{level}_s(t) \leq c_s^{max,level} \qquad \forall t \in T,  \forall s \in S$$


$$ x^{level}_s(1) = x_s^{level}(t_{e}) = 0.5 \cdot c_s^{max,level} \qquad \forall t \in T,  \forall s \in S$$ 

Of course, in addition the inflow/outflow of the storage also needs to be within the limit of the minimum and maximum power. 

$$ -c_s^{capacity} \leq x^{flow}_s(t) \leq c_s^{capacity} \qquad \forall t \in T, \forall s \in S$$ 


In [ ]:
es.add(
    fc.Storage(
        label="heat-storage",
        bus=heat_bus,
        carrier="water",
        tech="tank",
        capacity=heat_load_max * 4 * 0.1,
        storage_capacity=heat_load_max * 4,
        balanced=True, # oemof.solph argument
        initial_storage_level=0.5, # oemof.solph argument
        max_storage_level=1, # oemof.solph argument
    )
)

### Add heat pump and electro boiler 

To model the heat pump as well as the boiler, a conversion unit is used. This object will take from a bus and feed into another and is represented by the following mathematical equation: 

$$x^{flow}_{c, to}(t) = c^{efficiencty}_{c} \cdot x^{flow}_{c, from}(t), \qquad \forall c  \in C, \forall t \in T$$ 

The `capacity` refers to the output of the conversion unit. Thus, to the thermal capacity of the heat pump / boiler. Therefore the flow from the heat pump to the heating system is bounded by the thermal capacity of the repective unit:

$$x^{flow}_{c, to}(t) \leq c^{capacity}_{c, to}(t) \qquad \forall c  \in C, \forall t \in T$$



In [ ]:
es.add(
    fc.Conversion(
        label="heat-pump",
        from_bus=elec_bus,
        to_bus=heat_bus,
        carrier="electricity",
        tech="hp",
        capacity=heat_load_max * 0.5,
        efficiency=3
    )
)

es.add(
    fc.Conversion(
        label="electro-boiler",
        from_bus=elec_bus,
        to_bus=heat_bus,
        carrier="electricity",
        tech="p2h",
        capacity=heat_load_max * 0.3,
        efficiency=0.95
    )
)

### Objective Function 

The objective function is created from all instantiated objects. It will use all operating costs (i.e. `marginal_cost` argument) and if set all investment costs (i.e. `capacity_cost` argument)

$$ \text{min:} \sum_g \sum_t \overbrace{c^{marginal\_cost}_g \cdot x^{flow}_{g}(t)}^{\text{operating cost}} \\ 
\sum_g \sum_t \overbrace{c^{capacity\_cost}_g \cdot x^{capacity}_{g}(t)}^{\text{investment cost}} $$

**Note**: In this model the selling of electricity to the market is represented through negativ marginal cost of the excess component.

In [ ]:
m = Model(es)

# m.write(io_options={'symbolic_solver_labels': True})

In [ ]:
m.solve(solver="cbc")

m.results = m.results()


In [ ]:
pp.write_results(m, results_path)